In [ ]:
from lxml import etree
import pandas as pd

input_path = 'Nashville.xodr'


with open(input_path, 'r', encoding='utf-8') as f:
    original_text = f.read()
tree = etree.parse(input_path)
root = tree.getroot()

# Extract old/new edge IDs from <road>
old_edge_ids = []
new_edge_ids = []
warning_printed = False
for road in root.findall("road"):
    road_id = road.get("id")
    user_data = road.find("userData")
    if user_data is not None:
        sumo_id = user_data.get("value")

        if sumo_id.startswith("-"):
            new_id = f'{sumo_id[1:]}'
            old_edge_ids.append(road_id)
            new_edge_ids.append(new_id)
        else:
            if not warning_printed:
                print("To ensure Real-Twin works properly, please make all SUMO edge ID start with '-'")
                warning_printed = True



# Extract old/new junction IDs from <junction>
old_junction_ids = []
new_junction_ids = []

for junction in root.findall("junction"):
    junction_id = junction.get("id")
    junction_name = junction.get("name")
    old_junction_ids.append(junction_id)
    new_junction_ids.append(junction_name)


edgeID = pd.DataFrame({'oldedgeID': old_edge_ids, 'newedgeID': new_edge_ids})
junctionID = pd.DataFrame({'oldjunctionID': old_junction_ids, 'newjunctionID': new_junction_ids})

# Apply edge ID replacements
for _, row in edgeID.iterrows():
    old_id = row['oldedgeID']
    new_id = row['newedgeID']
    original_text = original_text.replace(f'id="{old_id}" junction=', f'id="{new_id}" junction=')
    original_text = original_text.replace(f'elementType="road" elementId="{old_id}"', f'elementType="road" elementId="{new_id}"')
    original_text = original_text.replace(f'incomingRoad="{old_id}"', f'incomingRoad="{new_id}"')
    original_text = original_text.replace(f'connectingRoad="{old_id}"', f'connectingRoad="{new_id}"')

# Apply junction ID replacements
for _, row in junctionID.iterrows():
    old_id = row['oldjunctionID']
    new_id = row['newjunctionID']
    original_text = original_text.replace(f'id="{old_id}">', f'id="{new_id}">')
    original_text = original_text.replace(f'elementType="junction" elementId="{old_id}"', f'elementType="junction" elementId="{new_id}"')
    original_text = original_text.replace(f'junction="{old_id}"', f'junction="{new_id}"')


with open(input_path, 'w', encoding='utf-8') as f:
    f.write(original_text)


